In [17]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli, Operator, random_statevector
from qiskit_aer import AerSimulator, Aer
import numpy as np
from scipy.linalg import sqrtm
import cvxpy as cp
import itertools
from IPython.display import clear_output
from winsound import Beep

def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def check_legal(matrix, print_errors=True):
    errors, legal = [], True
    if not hermitian(matrix):
        errors.append('not hermitian')
    if not trace_one(matrix):
        errors.append('trace not equal to one')
    if not positive_semi_definite(matrix):
        errors.append('not positive semidefinite')
    if len(errors) > 0:
        legal = False
    if print_errors:
        if not legal:
            print(f'input is not legal: ' + '; '.join(errors))
        else: 
            print('input is a legal density matrix')
    return legal
        
        
def generate_prob_lst(num_states):
    prob_lst = np.array([np.random.random() for _ in range(num_states)])
    prob_lst /= np.sum(prob_lst)
    return prob_lst

def get_rank(dm, tol=1e-10):
    return int(np.sum(np.linalg.eigvalsh(dm) > tol))

def get_fidelity(dm1, dm2, tol=1e-5):
    # assert is_legal(dm1) and is_legal(dm2), 'inputs are not legal density matrices'
    if not is_legal(dm1) and is_legal(dm2):
        print("Warning: inputs are not legal density matrices")
    try: 
        fidelity = (np.trace(sqrtm(sqrtm(dm1) @ dm2 @ sqrtm(dm1)))) ** 2
    except ValueError:
        print('fidelity cannot be computed for given inputs')
    assert np.abs(np.imag(fidelity)) < tol, 'fidelity is not real within tol'
    return fidelity.real

def generate_dm(num_qubits, num_states, state_lst=None, prob_lst=None):
    if state_lst is None:
        state_lst = [random_statevector(2**num_qubits) for _ in range(num_states)]
    if prob_lst is None:
        prob_lst = generate_prob_lst(num_states)
    density_matrix = sum([DensityMatrix(state_lst[i]).data * prob_lst[i] for i in range(num_states)])
    return density_matrix

def generate_random_Pauli_strings(num_strings, length, contain_I=True):
    if contain_I:
        characters = ['I', 'X', 'Y', 'Z']
    else:
        characters = ['X', 'Y', 'Z']
    generated_strings = []
    assert num_strings < len(characters) ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != 'I' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_random_01_strings(num_strings, length):
    characters = ['0', '1']
    generated_strings = []
    assert num_strings < 2 ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != '0' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_uv_Pauli_matrix(u_vec, v_vec):
    uv_map = {
        '00':'I', '01':'Z', '10':'X', '11':'Y'
    }
    Pauli_string = ''
    for u, v in zip(u_vec, v_vec):
        Pauli_char = uv_map.get(str(u) + str(v))
        if Pauli_char is None:
            raise ValueError('u or v list contains elements neither 0 or 1')
        else:
            Pauli_string += Pauli_char
    return Pauli_string

def generate_all_binary(length):
    all_strings = itertools.product('01', repeat=length)
    result = [''.join(s) for s in all_strings if '1' in s]
    return result

def generate_Pauli_expectations(dm, obsv):
    return np.trace(dm @ Pauli(obsv).to_matrix()).real

def get_trace_norm(dm):
    return np.sum(np.linalg.svd(dm, compute_uv=False))

In [23]:
def estimate_Pauli_expectations(dm, obsv, num_samples, simulation=False):
    num_samples = int(num_samples)
    if simulation: # simulate the process of sampling
        exp = np.real(np.trace(dm @ Pauli(obsv).to_matrix()))
        prob_p1 = (1 + exp) / 2
        prob_m1 = 1 - prob_p1
        samples = np.random.choice([+1, -1], size=num_samples, p=[prob_p1, prob_m1])
        return np.mean(samples)
    else: # use the approximate distribution instead
        exp = np.real(np.trace(dm @ Pauli(obsv).to_matrix()))
        num_samples_root = num_samples ** .5
        std_dev = (1 - exp ** 2) ** .5 / num_samples_root
        return np.random.normal(exp, std_dev)

In [24]:
# Prepare a state for measurement
num_qubit = 7
state = generate_dm(7, 5, prob_lst=np.array([.96, .01, .01, .01, .01]))
r = np.sum(np.linalg.eigvals(state) > 2e-2)
d = 2 ** num_qubit
num_samples = 1e10

# Prepare observables
c = 1
len_u_lst = int(c * r * (np.log(d)) ** 2)
v_lst = generate_all_binary(num_qubit)
u_lst = np.random.choice(v_lst, len_u_lst)
observables = [generate_uv_Pauli_matrix(u_vec, v_vec) for u_vec, v_vec in list(itertools.product(u_lst, v_lst))]
expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]

In [27]:
np.all(
    np.array(
        [np.abs(np.trace(state @ Pauli(o).to_matrix()) - e) <= 5e-5 for o, e in zip(observables, expectations)]
    )
)

np.True_

In [28]:
def optimize(dim, obsv, expct, tol=5e-5):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.abs(5 * cp.norm(sigma, 'nuc') + .5 * cp.norm(sigma, 'fro') ** 2))
    constraints = [cp.trace(sigma) == 1]
    for o, e in zip(obsv, expct):
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return sigma.value
    
    
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')
Beep(1000, 2000)

optimal
fidelity between outcome and target is 0.9922130030065852


In [13]:
with open('observables.txt', 'w') as file:
    for string in observables:
        file.write(f"{string}\n")

In [15]:
from scipy.io import savemat
savemat('array.mat', {'data': expectations})

In [ ]:
def optimize(dim, obsv, expct, tol=5e-2):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.sum([cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) ** 2 for o, e in zip(obsv, expct)]) + len(obsv) * cp.norm(sigma, 'nuc'))
    constraints = [sigma >> 0, cp.trace(sigma) == 1]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return problem.status, sigma.value
    
    
status, sigma = optimize(d, observables, expectations)
if status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    Beep(3000, 2000) 
    raise ValueError('optimization failed')
else:
    check_legal(sigma, print_errors=True)
    print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')
    Beep(1000, 2000)

c:\Users\Neville\Documents\Quantum Computing 3\Entanglement Detection\.venv\Lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "
